In [ ]:


## STEP 1: Set up the workspace.

# import json
import os
from dotenv import load_dotenv
load_dotenv()

my_secret = os.getenv("MY_SECRET")

#GETTING MY ID:
#id = open('id_CONFIDENTIAL.json', 'r')
#mi = json.load(id)
my_id = my_secret

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = my_id,
                      resource_group = "resource_group2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

